#### Original Paper

Turner et al. (2015). **Data Analysis for Strength and conditioning coaches: Using excel to analyze reliability, differences, and relationships.** *Strength Cond J*; 37(1): 76-83.

https://www.researchgate.net/publication/272507171_Data_Analysis_for_Strength_and_Conditioning_Coaches_Using_Excel_to_Analyze_Reliability_Differences_and_Relationships

#### Blog Post

http://optimumsportsperformance.com/blog/doing-things-in-python-that-youd-normally-do-in-excel-data-analysis-for-strength-conditioning-coaches-turner-et-al-2015/

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import math

## Create Data from Paper

In [34]:
dat = pd.DataFrame()
dat['athlete'] = np.arange(start = 1, stop = 25, step = 1)
dat['test_session'] = np.repeat(a = 1, repeats = 24)
dat['cmj_1'] = [38.1,34.1,42.8,50.4,37,42.1,44.5,37.9,43.6,35.4,45.3,46.6,30.2,50.8,41,43.7,43.6,43.6,51.6,27.8,29,29,30.3,34.9]
dat['cmj_2'] = [43.6,35.7,42.2,52.2,36.1,40.7,42.3,33.3,44.9,40.1,44.4,47.7,31,51.2,46.7,40.4,44.1,44.8,50.4,28.5,28.3,27.7,35.7,36.1]
dat['cmj_3'] = [47.7,36,41.9,52.2,36.9,45.1,45.2,39.6,37.4,39.5,43.9,47.7,31.5,53.7,36.4,43,48.9,43.5,61.6,28.7,30.9,26.3,34.2,35.4]

In [35]:
dat.head(10)

,athlete,test_session,cmj_1,cmj_2,cmj_3
0,1,1,38.1,43.6,47.7
1,2,1,34.1,35.7,36.0
2,3,1,42.8,42.2,41.9
3,4,1,50.4,52.2,52.2
4,5,1,37.0,36.1,36.9
5,6,1,42.1,40.7,45.1
6,7,1,44.5,42.3,45.2
7,8,1,37.9,33.3,39.6
8,9,1,43.6,44.9,37.4
9,10,1,35.4,40.1,39.5


## Explore Basic Features of the Data

In [22]:
# Note that we created a pandas data frame

type(dat)

pandas.core.frame.DataFrame

In [23]:
# How many rows?

len(dat)

24

In [24]:
# Rows x Columns

dat.shape

(24, 5)

In [25]:
# Column names

dat.columns

Index(['athlete', 'test_session', 'cmj_1', 'cmj_2', 'cmj_3'], dtype='object')

## Get Mean & SD from Each CMJ Column

**1) Define a function for calculating our summary stats with one line of code**

**2) Confirm that the stats are the same as that which is reported in the paper**

In [26]:
## function for summary stats

def summary_stats(val):
    
    avg = val.mean()
    sd = val.std()
    
    results = [["CMJ Avg", round(avg, 1)], ["CMJ SD", round(sd, 1)]]
    
    return results

In [27]:
# CMJ 1

summary_stats(dat['cmj_1'])

[['CMJ Avg', 39.7], ['CMJ SD', 7.2]]

In [28]:
# CMJ 2

summary_stats(dat['cmj_2'])

[['CMJ Avg', 40.3], ['CMJ SD', 7.2]]

In [29]:
# CMJ 3

summary_stats(dat['cmj_3'])

[['CMJ Avg', 41.1], ['CMJ SD', 8.4]]

## Calculate Smallest Worthwhile Change (SWC) and Typical Error of Measurement (TEM)

In the paper, Anthony Turner adds the values to each athlete within the same row, rather than creating a separate data frame for SWC and TEM. So we will structure the data in the same manner as he did.

In [36]:
# First create a column to identify each athlete's max value across the 3 jumps
# NOTE: specify that you are performing the max calculation across rows by indicating axis = 1

dat['best_cmj'] = dat.iloc[:,2:5].max(axis = 1)
dat.head()

,athlete,test_session,cmj_1,cmj_2,cmj_3,best_cmj
0,1,1,38.1,43.6,47.7,47.7
1,2,1,34.1,35.7,36.0,36.0
2,3,1,42.8,42.2,41.9,42.8
3,4,1,50.4,52.2,52.2,52.2
4,5,1,37.0,36.1,36.9,37.0


In [40]:
# Calculate a column for each athletes Average, SD, & CV% of CMJ

dat['avg'] = round(dat.loc[:,['cmj_1', 'cmj_2', 'cmj_3']].mean(axis = 1), 1)
dat['sd'] = round(dat.loc[:,['cmj_1', 'cmj_2', 'cmj_3']].std(axis = 1), 1)
dat['cv_pct'] = round(dat['sd'] / dat['avg'], 4)

dat.head()

,athlete,test_session,cmj_1,cmj_2,cmj_3,best_cmj,avg,sd,cv_pct
0,1,1,38.1,43.6,47.7,47.7,43.1,4.8,0.1114
1,2,1,34.1,35.7,36.0,36.0,35.3,1.0,0.0283
2,3,1,42.8,42.2,41.9,42.8,42.3,0.5,0.0118
3,4,1,50.4,52.2,52.2,52.2,51.6,1.0,0.0194
4,5,1,37.0,36.1,36.9,37.0,36.7,0.5,0.0136


In [38]:
## Calculate SWC
# Turner uses 0.2 * Btw Subject Standard Deviation of their best CMJ (per Hopkins)

effect_of_interest = 0.2
swc = round(effect_of_interest * dat['best_cmj'].std(), 2)
swc

1.63

In [39]:
## calculate typical error measurement as a CV%
# Turner takes the average of all of the athletes CV% across their 3 jumps

tem_cv = round(dat['cv_pct'].mean(), 3)
tem_cv

0.049

## Add the SWC and TEM to each row for each athlete to specify training targets

In [41]:
## Add the SWC to each athletes Best CMJ to identify a goal for the next round of testing

dat['goal_swc'] = swc + dat['best_cmj']
dat.head()

,athlete,test_session,cmj_1,cmj_2,cmj_3,best_cmj,avg,sd,cv_pct,goal_swc
0,1,1,38.1,43.6,47.7,47.7,43.1,4.8,0.1114,49.33
1,2,1,34.1,35.7,36.0,36.0,35.3,1.0,0.0283,37.63
2,3,1,42.8,42.2,41.9,42.8,42.3,0.5,0.0118,44.43
3,4,1,50.4,52.2,52.2,52.2,51.6,1.0,0.0194,53.83
4,5,1,37.0,36.1,36.9,37.0,36.7,0.5,0.0136,38.63


In [42]:
## Add the error measurement to each athletes Best CMJ

dat['error_measurement'] = tem_cv * dat['best_cmj']
dat.head()

,athlete,test_session,cmj_1,cmj_2,cmj_3,best_cmj,avg,sd,cv_pct,goal_swc,error_measurement
0,1,1,38.1,43.6,47.7,47.7,43.1,4.8,0.1114,49.33,2.3373
1,2,1,34.1,35.7,36.0,36.0,35.3,1.0,0.0283,37.63,1.7640
2,3,1,42.8,42.2,41.9,42.8,42.3,0.5,0.0118,44.43,2.0972
3,4,1,50.4,52.2,52.2,52.2,51.6,1.0,0.0194,53.83,2.5578
4,5,1,37.0,36.1,36.9,37.0,36.7,0.5,0.0136,38.63,1.8130


In [44]:
# One issue is that the SWC is a small fraction (0.2) of the between subject SD
# We could use a larger magnitude than 0.2.
# For example, we can adjust the swc to be 0.6 instead of 0.2

dat['adj_swc'] = 0.6 * dat['best_cmj'].std() + dat['best_cmj']
dat.head()

,athlete,test_session,cmj_1,cmj_2,cmj_3,best_cmj,avg,sd,cv_pct,goal_swc,error_measurement,adj_swc
0,1,1,38.1,43.6,47.7,47.7,43.1,4.8,0.1114,49.33,2.3373,52.59769
1,2,1,34.1,35.7,36.0,36.0,35.3,1.0,0.0283,37.63,1.7640,40.89769
2,3,1,42.8,42.2,41.9,42.8,42.3,0.5,0.0118,44.43,2.0972,47.69769
3,4,1,50.4,52.2,52.2,52.2,51.6,1.0,0.0194,53.83,2.5578,57.09769
4,5,1,37.0,36.1,36.9,37.0,36.7,0.5,0.0136,38.63,1.8130,41.89769


**NOTE:** Now the improvement for some athletes that already have a high CMJ at baseline might be untainable, as an improvement for them (given their starting athleticism) might be smaller. Care should be taken when determining such training targets. The heterogeneity of the group also needs to be considered given that the between subject SD was used in the calculation of SWC.

## Show group performance improvements using Cohen's d

Since our data is structured as 3 CMJ's performed in a single session, we will just select jumps 1 and 2 and *pretend* that they are representing an initial CMJ and then a CMJ followed up after a period of training.

In [46]:
dat_pre_post = dat.iloc[:, 0:4]
dat_pre_post.head()

,athlete,test_session,cmj_1,cmj_2
0,1,1,38.1,43.6
1,2,1,34.1,35.7
2,3,1,42.8,42.2
3,4,1,50.4,52.2
4,5,1,37.0,36.1


In [47]:
## get summary stats of the two jumping periods

avg_pre = dat_pre_post['cmj_1'].mean()
sd_pre = dat_pre_post['cmj_1'].std()

avg_post = dat_pre_post['cmj_2'].mean()
sd_post = dat_pre_post['cmj_2'].std()

In [48]:
print(avg_pre,
      sd_pre,
      avg_post,
      sd_post)

39.72083333333333 7.2086288370358576 40.337500000000006 7.236910973252512


In [49]:
# calculate mean difference

mean_diff = avg_post - avg_pre
mean_diff

0.6166666666666742

In [50]:
# calculate pooed SD
# NOTE: To square the standard deviations in Python use the ** operator instead of the ^ symbol, which is commonly used

sd_pooled = math.sqrt((sd_post**2 + sd_pre**2) / 2)
sd_pooled

7.22278374814474

In [51]:
# calculate Cohen's d Effect Size

ES = mean_diff / sd_pooled
ES

0.08537797726881587

**The difference between jump 1 and jump 2, for the group, was found to be trival.**

In [52]:
es_interpretation = pd.DataFrame()
es_interpretation['Effect Size Statistic'] = ['< 0.2', '0.2 - 0.6', '0.6 - 1.2', '1.2 - 2.0', '2.0 - 4.0', '> 4.0']
es_interpretation['Qualitative Descriptor'] = ['Trivial', 'Small', 'Moderate', 'Large', 'Very Large', 'Extremely Large']

es_interpretation

,Effect Size Statistic,Qualitative Descriptor
0,< 0.2,Trivial
1,0.2 - 0.6,Small
2,0.6 - 1.2,Moderate
3,1.2 - 2.0,Large
4,2.0 - 4.0,Very Large
5,> 4.0,Extremely Large
